# AutoGluon-Text-Classificationを用いたテキスト分類

## 概要
このノートブックでは、AutoGluon-Text-Classificationを用いて日本語のAmazonの商品レビューに対する感情分析を行います。
つまり、そのレビューが Positive (Rating が 5 or 4) か、Negative (Rating が 1 or 2)なのかを判定します。
これは文書を Positive か Negative に分類する2クラスの分類問題として扱うことができます。

AutoGluon-Text-Classificationは、データを用意し、学習時にパラメータを設定するだけでモデルを作成することができます。学習には多くのメモリを必要としますので、多くのメモリを利用できるインスタンスを利用してください (c5.9xlarge インスタンスで検証済みです) 

## 準備
必要となるライブラリをインストールします。

In [ ]:
!pip install --upgrade mxnet-cu100
!pip install autogluon
!pip install gluonnlp==0.8.1

AutoGluonライブラリをインポートします。今回はTextClassificationを利用します。

In [ ]:
from autogluon import TextClassification as task

## データの取得

Amazon の商品レビューデータセットは [Registry of Open Data on AWS](https://registry.opendata.aws/) で公開されており、 
以下からダウンロード可能です。
このノートブックでは、日本語のデータセットをダウンロードします。
- データセットの概要  
https://registry.opendata.aws/amazon-reviews/

- 日本語のデータセット(readme.htmlからたどることができます）  
https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz

以下では、データをダウンロードして解凍 (unzip) します。

In [ ]:
import urllib.request
import os
import gzip
import shutil

download_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz" 
dir_name = "data"
file_name = "amazon_review.tsv.gz"
tsv_file_name = "amazon_review.tsv"
file_path = os.path.join(dir_name,file_name)
tsv_file_path = os.path.join(dir_name,tsv_file_name)

os.makedirs(dir_name, exist_ok=True)

if os.path.exists(file_path):
    print("File {} already exists. Skipped download.".format(file_name))
else:
    urllib.request.urlretrieve(download_url, file_path)
    print("File downloaded: {}".format(file_path))
    
if os.path.exists(tsv_file_path):
    print("File {} already exists. Skipped unzip.".format(tsv_file_name))
else:
    with gzip.open(file_path, mode='rb') as fin:
        with open(tsv_file_path, 'wb') as fout:
            shutil.copyfileobj(fin, fout)
            print("File uznipped: {}".format(tsv_file_path))

## データの前処理

ダウンロードしたデータには学習に不要なデータや直接利用できないデータもあります。以下の前処理で利用できるようにします。

1. ダウンロードしたデータには不要なデータも含まれているので削除し、2クラス分類 (positive が 1, negative が 0)となるように評価データを加工します。
2. 学習データ、テストデータに分けて、保存します。

### データの加工

今回利用しないデータは以下の2つです。必要なデータだけ選んで保存します。
- 評価データ `star_rating` と レビューのテキストデータ `review_body` 以外のデータ
- 評価が 3 のデータ (positive でも negative でもないデータ)

また、評価が1, 2 のデータはラベル 0 (negative) に、評価が4, 5 のデータはラベル 1 (positive) にします。

In [ ]:
import pandas as pd
df = pd.read_csv(tsv_file_path, sep ='\t')
df_pos_neg = df.loc[:, ["star_rating", "review_body"]]
df_pos_neg = df_pos_neg[df_pos_neg.star_rating != 3]
df_pos_neg.loc[df_pos_neg.star_rating < 3, "star_rating"] = 0
df_pos_neg.loc[df_pos_neg.star_rating > 3, "star_rating"] = 1

### データの分割

すべてのデータを学習データとすると、データを使って作成したモデルが良いのか悪いのか評価するデータが別途必要になります。
そこで、データを学習データ、テストデータに分割して利用します。学習データはモデルの学習に利用し、最終的に作成されたモデルに対してテストデータによる評価を行います。

`train_ratio` で設定した割合のデータを学習データとし、残ったデータをテストデータに分割して利用します。

In [ ]:
import numpy as np
import os
dir_name = "data"
file_name = "amazon_review.tsv.gz"
tsv_file_name = "amazon_review.tsv"
file_path = os.path.join(dir_name,file_name)
tsv_file_path = os.path.join(dir_name,tsv_file_name)

# Swap positions of "review_body","star_rating" because transform.py requires this order.
labeled_df = df_pos_neg.loc[:, ["review_body","star_rating"]]
data_size = len(labeled_df.index)
train_ratio = 0.9
train_index = np.random.choice(data_size, int(data_size*train_ratio), replace=False)
test_index = np.setdiff1d(np.arange(data_size), train_index)

np.savetxt('train.tsv',labeled_df.iloc[train_index].values, fmt="%s\t%i") 
np.savetxt('test.tsv',labeled_df.iloc[train_index].values, fmt="%s\t%i") 

print("Data is splitted into:")
print("Training data: {} records.".format(len(train_index)))
print("Test data: {} records.".format(len(test_index)))

### csvファイルへ変換
tsvファイルからcsvファイルへ変換し、ラベル名textとtargetを追加します。
このノートブックでは試験的な利用のため、トレーニングのデータを1000件、テストデータを100件として、短い計算で終えるようにします。

In [ ]:
!python -c 'import sys, csv; csv.writer(sys.stdout, delimiter=",").writerows(csv.reader(sys.stdin, delimiter="\t"))' < train.tsv > tmp.csv
!cat tmp.csv | sed "1itext,target" > train.csv
!head -1001 train.csv > train_1000.csv

!python -c 'import sys, csv; csv.writer(sys.stdout, delimiter=",").writerows(csv.reader(sys.stdin, delimiter="\t"))' < test.tsv > tmp2.csv
!cat tmp2.csv | sed "1itext,target" > test.csv
!head -101 test.csv > test_100.csv

## 学習の実行

### 学習データの指定
filepathにデータのパス、usecolsに学習のペアとなるラベルを指定します。

In [ ]:
dataset = task.Dataset(filepath='train_1000.csv', usecols=['text', 'target'])

### 学習
AutoGluonのテキスト分類では、予めいくつかの事前学習済みBERTモデルを使用することができます。
今回は日本語のレビューなので、wikipediaの複数言語のテキストで学習されたモデルである`wiki_multilingual_uncased`を使用します。

In [ ]:
predictor = task.fit(dataset, epochs=1,pretrained_dataset='wiki_multilingual_uncased')

学習の過程においてどのような探索を行ったか、確認してみましょう。

In [ ]:
results = predictor.fit_summary()

## 推論

### 評価
バリデーションセットのaccuracyを以下で確認できます。

In [ ]:
print('Top-1 val acc: %.3f' % predictor.results['best_reward'])

テストセットを読み込み、accuracyを以下で確認できます。

In [ ]:
testset = task.Dataset(filepath='test_100.csv', usecols=['text', 'target'])
test_acc = predictor.evaluate(testset)
print('Top-1 test acc: %.3f' % test_acc)

### 推論
実際の文書を用いた推論も行えます。

In [ ]:
sentence = '子供でも簡単に操作ができて、暇つぶしに最高です。'
ind = predictor.predict(sentence)
print('The input sentence sentiment is classified as [%d].' % ind.asscalar())